# Get a list of all dataframes

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime


#Get all months from Jan 22 until today
datelist = pd.date_range(start ='01/01/2022' , end = '31/08/2023',freq="M").tolist()
datelist = [x.strftime('%Y-%m') for x in datelist]
#print(datelist)

def get_all_dataframes(datelist):
    
    df_list=[]

    for month_d in datelist:

        month_parq = month_d +'.parquet'
        URL_DATA = 'https://storage.googleapis.com/dosm-public-pricecatcher/pricecatcher_{}'.format(month_parq)
        
        df = pd.read_parquet(URL_DATA)
        if 'date' in df.columns: df['date'] = pd.to_datetime(df['date'])
        #df.head()
        df_list.append(df)
    
    return df_list

#This stores all dataframes since Jan 22
df_all_list = get_all_dataframes(datelist)
print(f'Number of dataframes in the list: {len(df_all_list)}')


Number of dataframes in the list: 20


# Intersections

In [2]:

df_items_unique = pd.DataFrame(columns=datelist)
i=0

list2_df = df_all_list[0]

list2 = list2_df["item_code"].unique()
print(f'initial len list2 {len(list2)}')
for x_df in df_all_list:
    list_of_items = x_df["item_code"].unique() #get the list of items for that month


    #Get Intersections of ALL lists
    list2 = list(set(list_of_items) & set(list2))
    print(len(list2))

    #Putting in dataframes-we dont need this, just for recording purposes  
    #new_col = pd.DataFrame(list_of_items,columns=["item_code"])
    #df_items_unique[datelist[i]]=new_col
    #i+=1

#df_items_unique.head()


#Actually, come to think of it, we don't need to find the intersections. We just need to select the items in the current month
# and force other months to follow 

initial len list2 724
724
694
693
502
502
502
502
501
501
501
492
491
491
490
490
490
490
490
490
490


## df_new is a list of all items(by item_code) at the intersection

In [3]:
list2
list2 = [x.astype(np.int64) for x in list2]
print(len(list2))
print(type(list2[0]))

df_new = pd.DataFrame(list2,columns=['item_code'])
df_new.info()
df_new.head()


#df_new is a list of all items at the intersection

490
<class 'numpy.int64'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490 entries, 0 to 489
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   item_code  490 non-null    int64
dtypes: int64(1)
memory usage: 4.0 KB


,item_code
0,1
1,2
2,3
3,9
4,10


In [4]:
URL_DATA = 'https://storage.googleapis.com/dosm-public-pricecatcher/lookup_item.parquet'
df_item_code = pd.read_parquet(URL_DATA)
if 'date' in df_item_code.columns: df_item_code['date'] = pd.to_datetime(df_item_code['date'])

df_item_code.info()
df_item_code.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 757 entries, 0 to 756
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   item_code      757 non-null    int64 
 1   item           756 non-null    object
 2   unit           756 non-null    object
 3   item_group     756 non-null    object
 4   item_category  756 non-null    object
dtypes: int64(1), object(4)
memory usage: 29.7+ KB


,item_code,item,unit,item_group,item_category
0,-1,None,None,None,None
1,1,AYAM BERSIH - STANDARD,1kg,BARANGAN SEGAR,AYAM
2,2,AYAM BERSIH - SUPER,1kg,BARANGAN SEGAR,AYAM
3,3,AYAM HIDUP,1kg,BARANGAN SEGAR,AYAM
4,9,DAGING KAMBING BEBIRI IMPORT BERTULANG (MUTTON...,1kg,BARANGAN SEGAR,DAGING


In [5]:
x = df_new.iloc[0]['item_code']
print(type(x))
print(x)

<class 'numpy.int64'>
1


In [6]:
y = df_item_code.iloc[1]['item_code']
print(type(y))

print('yesss' if x==y else 'noo')

<class 'numpy.int64'>
yesss


In [15]:

tempdf = df_new.copy() #must make a copy . doing tempdf=df7 is just reassigning
u = tempdf.pop('item_code').map(df_item_code.set_index('item_code')['item'])
#u.name = "item"

print(u.size)
u.head()


490


0                               AYAM BERSIH - STANDARD
1                                  AYAM BERSIH - SUPER
2                                           AYAM HIDUP
3    DAGING KAMBING BEBIRI IMPORT BERTULANG (MUTTON...
4    DAGING KAMBING BEBIRI IMPORT BERTULANG (MUTTON...
Name: item_code, dtype: object

In [31]:
print(df_new.size)
df_new.tail()

490


,item_code
485,2003
486,2004
487,2008
488,2009
489,2010


# This is the list of intsection items from Jan 22 until now using method 1 (intersection)

In [34]:
#df_new2 = pd.concat([df_new,u])
#df_new2.head()


df_new2 = pd.merge(df_new,df_item_code,how="left",on="item_code")
print(len(df_new2.index))
df_new2.tail()

490


,item_code,item,unit,item_group,item_category
485,2003,BIHUN KERING IMPORT (PELBAGAI JENAMA),400 g,BARANGAN BERBUNGKUS,BIHUN
486,2004,BERAS SUPER SPECIAL CAP FAMILI 5%,10 kg,BARANGAN BERBUNGKUS,BERAS
487,2008,SUSU TEPUNG PENUH KRIM SEGERA NESPRAY,480 g,BARANGAN BERBUNGKUS,KRIMER DAN SUSU TEPUNG
488,2009,DUTCH LADY 123 (BIASA),850g,SUSU DAN BARANGAN BAYI,SUSU BAYI
489,2010,DUTCH LADY 456 (BIASA),850g,SUSU DAN BARANGAN BAYI,SUSU BAYI


# Method 2- Just get the list of items from the current month and work backwards

# Get the price of all common items

In [ ]:
df_items_unique = pd.DataFrame(columns=datelist)

list2_df = df_all_list[0]

list2 = list2_df["item_code"].unique()
print(f'initial len list2 {len(list2)}')

for x_df in df_all_list:



    list_of_items = x_df["item_code"].unique() #get the list of items for that month


    #Get Intersections of ALL lists
    list2 = list(set(list_of_items) & set(list2))
    print(len(list2))
